In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [2]:
dataset = pd.read_csv('Google_Stock_Price_Train.csv',index_col="Date",parse_dates=True)

In [3]:
dataset.head()

,Open,High,Low,Close,Volume
Date,,,,,
2012-01-03,325.25,332.83,324.97,663.59,"7,380,500"
2012-01-04,331.27,333.87,329.08,666.45,"5,749,400"
2012-01-05,329.83,330.75,326.89,657.21,"6,590,300"
2012-01-06,328.34,328.77,323.68,648.24,"5,405,900"
2012-01-09,322.04,322.29,309.46,620.76,"11,688,800"


In [4]:
dataset["Close"] = dataset["Close"].str.replace(',', '').astype(float)

In [5]:
dataset["Volume"] = dataset["Volume"].str.replace(',', '').astype(float)

In [6]:
# 7 day rolling mean
dataset.rolling(7).mean().head(20)

,Open,High,Low,Close,Volume
Date,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN
2012-01-04,NaN,NaN,NaN,NaN,NaN
2012-01-05,NaN,NaN,NaN,NaN,NaN
2012-01-06,NaN,NaN,NaN,NaN,NaN
2012-01-09,NaN,NaN,NaN,NaN,NaN
2012-01-10,NaN,NaN,NaN,NaN,NaN
2012-01-11,323.002857,325.392857,318.682857,643.132857,7.208100e+06
2012-01-12,321.457143,322.882857,316.841429,638.037143,6.691514e+06
2012-01-13,318.698571,319.801429,314.025714,631.870000,6.531857e+06


In [7]:
training_set=dataset['Open']
training_set=pd.DataFrame(training_set)

In [8]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [9]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [10]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [11]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 10, batch_size = 32)
#regressor.score(X_train, y_train)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1198/1198 [==============================] - 6s 5ms/step - loss: 0.0570
Epoch 2/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0076
Epoch 3/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0054
Epoch 4/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0054
Epoch 5/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0052
Epoch 6/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0046
Epoch 7/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0046
Epoch 8/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0051
Epoch 9/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0042
Epoch 10/10
1198/1198 [==============================] - 4s 3ms/step - loss: 0.0043


In [35]:
from sklearn.metrics import mean_squared_error

predicted = regressor.predict(np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)))
predicted = np.array([i[0] for i in predicted])
print("score on prediction : ", 1 - mean_squared_error(y_train, predicted))

score on prediction :  0.9982676835517655


In [13]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [14]:
from sklearn.linear_model import LinearRegression

In [32]:
#linear regression
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)
confidence = clf.score(X_train[:100], y_train[:100])
print('ACCURACY -> ', confidence)

ACCURACY ->  0.9115054302265969
